# dependencies
connect to cpu if want to use behaviour cloning


In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 9.5 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.

# environment and expert

In [4]:
import gymnasium as gym
import imitation
import seals

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms import bc
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
import torch
from torch.optim.adam import Adam
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def make_env_expert_transitions(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
        "seals/Walker2d-v1", # seals:seals/CartPole-v0
        rng=np.random.default_rng(SEED),
        n_envs=n_trajs,
        post_wrappers=[
            lambda env, _: RolloutInfoWrapper(env)
        ],  # needed for computing rollouts later
        parallel=True
    )
    expert = load_policy(
        "ppo-huggingface",
        organization="HumanCompatibleAI",
        env_name="seals/Walker2d-v1",
        venv=env,
    )
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=1000, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories
    )
    transitions = rollout.flatten_trajectories(rollouts)

    return env, expert, transitions


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
from typing import Callable
from stable_baselines3 import PPO
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [8]:
def make_BC_trainer(env, expert, transitions):
    global SEED
# 可以调policy network的架构
# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[dict(pi=[32, 32], vf=[32, 32])])
    policy_network = MlpPolicy(observation_space = env.observation_space,
                            action_space = env.action_space,
                            lr_schedule = linear_schedule(0.0003034620018780926))

    bc_trainer = bc.BC(
        observation_space=env.observation_space,
        action_space=env.action_space,
        policy = policy_network, # default optimizer is adam
        batch_size = 32,
        optimizer_kwargs = {"lr": 0.0003034620018780926},
        demonstrations=transitions,
        rng=np.random.default_rng(SEED),
    )
    return bc_trainer

# BC - 1 trajectory

In [9]:
env_1, expert_1, transitions_1 = make_env_expert_transitions(1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-seals-Walker2d-v1.zip:   0%|          | 0.00/1.75M [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
bc_trainer_1 = make_BC_trainer(env_1, expert_1, transitions_1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2408.5057490499994 +/- 430.2184107970064


In [12]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_1.policy, env_1, 20, return_episode_rewards=True
)

In [13]:
env_1.seed(SEED)
bc_trainer_1.train(n_epochs = 60, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 165      |
|    loss           | 8.05     |
|    neglogp        | 8.06     |
|    prob_true_act  | 0.000357 |
|    samples_so_far | 32       |
--------------------------------


12batch [00:00, 110.82batch/s]
51batch [00:00, 166.47batch/s]
90batch [00:00, 182.46batch/s]
110batch [00:00, 186.03batch/s]
151batch [00:00, 185.53batch/s]
171batch [00:00, 189.04batch/s]
211batch [00:01, 192.51batch/s]
231batch [00:01, 190.38batch/s]
272batch [00:01, 190.17batch/s]
292batch [00:01, 188.01batch/s]
332batch [00:01, 192.06batch/s]
372batch [00:02, 188.42batch/s]
391batch [00:02, 185.11batch/s]
430batch [00:02, 184.32batch/s]
449batch [00:02, 179.46batch/s]
490batch [00:02, 188.21batch/s]
Epoch 15 of 60                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00745 |
|    entropy        | 7.45     |
|    epoch          | 16       |
|    l2_loss        | 0        |
|    l2_norm        | 172      |
|    loss           | 5.17     |
|    neglogp        | 5.18     |
|    prob_true_act  | 0.00719  |
|    samples_so_far | 16032    |
--------------------------------


509batch [00:02, 172.21batch/s]
548batch [00:03, 177.16batch/s]
584batch [00:03, 176.70batch/s]
602batch [00:03, 175.33batch/s]
643batch [00:03, 188.57batch/s]
682batch [00:03, 185.73batch/s]
701batch [00:03, 179.16batch/s]
741batch [00:04, 188.17batch/s]
760batch [00:04, 186.23batch/s]
800batch [00:04, 191.99batch/s]
820batch [00:04, 191.87batch/s]
861batch [00:04, 193.03batch/s]
881batch [00:04, 187.46batch/s]
921batch [00:05, 182.19batch/s]
959batch [00:05, 182.50batch/s]
978batch [00:05, 176.37batch/s]
996batch [00:05, 171.32batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 1000     |
|    ent_loss       | -0.00651 |
|    entropy        | 6.51     |
|    epoch          | 32       |
|    l2_loss        | 0        |
|    l2_norm        | 176      |
|    loss           | 3.96     |
|    neglogp        | 3.96     |
|    prob_true_act  | 0.0209   |
|    samples_so_far | 32032    |
--------------------------------


1015batch [00:05, 175.47batch/s]
1053batch [00:05, 179.12batch/s]
1071batch [00:05, 169.09batch/s]
1110batch [00:06, 177.57batch/s]
1128batch [00:06, 174.59batch/s]
1170batch [00:06, 190.40batch/s]
1190batch [00:06, 178.85batch/s]
1230batch [00:06, 185.67batch/s]
1270batch [00:06, 189.51batch/s]
1290batch [00:07, 191.59batch/s]
1332batch [00:07, 197.10batch/s]
1352batch [00:07, 189.16batch/s]
1392batch [00:07, 191.53batch/s]
1412batch [00:07, 183.18batch/s]
1450batch [00:07, 179.29batch/s]
1487batch [00:08, 178.75batch/s]
Epoch 47 of 60                  

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 1500     |
|    ent_loss       | -0.0056  |
|    entropy        | 5.6      |
|    epoch          | 48       |
|    l2_loss        | 0        |
|    l2_norm        | 181      |
|    loss           | 3.16     |
|    neglogp        | 3.17     |
|    prob_true_act  | 0.0478   |
|    samples_so_far | 48032    |
--------------------------------


1505batch [00:08, 168.00batch/s]
1545batch [00:08, 181.61batch/s]
1564batch [00:08, 178.84batch/s]
1602batch [00:08, 180.25batch/s]
1639batch [00:09, 171.46batch/s]
1657batch [00:09, 170.62batch/s]
1696batch [00:09, 180.17batch/s]
1735batch [00:09, 178.93batch/s]
1753batch [00:09, 155.39batch/s]
1785batch [00:10, 132.95batch/s]
1826batch [00:10, 125.19batch/s]
1853batch [00:10, 123.54batch/s]
1860batch [00:10, 174.44batch/s]


In [14]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_1.policy, env_1, 20, return_episode_rewards=True
)

In [15]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: 83.05898515 +/- 2.417199728003031


In [16]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 1962.1633954000001 +/- 448.2051063422452


In [17]:
import torch
torch.save(bc_trainer_1.policy, "BC_MlpPolicy_Walker2D_1.pth.tar")

# BC - 4 trajectory

In [18]:
env_4, expert_4, transitions_4 = make_env_expert_transitions(4)

In [19]:
bc_trainer_4 = make_BC_trainer(env_4, expert_4, transitions_4)

In [20]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2565.5421143 +/- 280.4971151875886


In [21]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_4.policy, env_4, 20, return_episode_rewards=True
)

In [22]:
env_4.seed(SEED)
bc_trainer_4.train(n_epochs = 80, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 165      |
|    loss           | 8.1      |
|    neglogp        | 8.1      |
|    prob_true_act  | 0.000339 |
|    samples_so_far | 32       |
--------------------------------


119batch [00:00, 198.40batch/s]
241batch [00:01, 199.03batch/s]
369batch [00:01, 198.40batch/s]
488batch [00:02, 193.85batch/s]
Epoch 3 of 80                  

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00746 |
|    entropy        | 7.46     |
|    epoch          | 4        |
|    l2_loss        | 0        |
|    l2_norm        | 170      |
|    loss           | 4.89     |
|    neglogp        | 4.9      |
|    prob_true_act  | 0.00813  |
|    samples_so_far | 16032    |
--------------------------------


607batch [00:03, 186.90batch/s]
731batch [00:03, 200.03batch/s]
873batch [00:04, 194.53batch/s]
992batch [00:05, 194.74batch/s]
Epoch 7 of 80                  

...

8610batch [00:49, 205.24batch/s]
8749batch [00:50, 178.56batch/s]
8869batch [00:50, 180.94batch/s]
8986batch [00:51, 187.54batch/s]
Epoch 71 of 80                  

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 9000     |
|    ent_loss       | 0.000467 |
|    entropy        | -0.467   |
|    epoch          | 72       |
|    l2_loss        | 0        |
|    l2_norm        | 220      |
|    loss           | -0.659   |
|    neglogp        | -0.66    |
|    prob_true_act  | 15.6     |
|    samples_so_far | 288032   |
--------------------------------


9119batch [00:52, 182.97batch/s]
9238batch [00:52, 191.47batch/s]
9359batch [00:53, 195.69batch/s]
9481batch [00:54, 193.85batch/s]
Epoch 75 of 80                  

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 9500     |
|    ent_loss       | 0.000512 |
|    entropy        | -0.512   |
|    epoch          | 76       |
|    l2_loss        | 0        |
|    l2_norm        | 222      |
|    loss           | -1       |
|    neglogp        | -1       |
|    prob_true_act  | 12.8     |
|    samples_so_far | 304032   |
--------------------------------


9624batch [00:54, 190.72batch/s]
9745batch [00:55, 196.74batch/s]
9870batch [00:56, 197.08batch/s]
9991batch [00:56, 193.03batch/s]
10000batch [00:56, 175.75batch/s]


In [23]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_4.policy, env_4, 20, return_episode_rewards=True
)

In [24]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: 121.83767819999998 +/- 1.364413792919163


In [25]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 2428.99631625 +/- 239.23365453747877


In [26]:
import torch
torch.save(bc_trainer_4.policy, "BC_MlpPolicy_Walker2D_4.pth.tar")

# BC - 10 trajectory

In [27]:
env_10, expert_10, transitions_10 = make_env_expert_transitions(10)

In [28]:
bc_trainer_10 = make_BC_trainer(env_10, expert_10, transitions_10)

In [29]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2496.4408684500004 +/- 213.94912036353224


In [30]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_10.policy, env_10, 20, return_episode_rewards=True
)

In [31]:
env_10.seed(SEED)
bc_trainer_10.train(n_epochs = 50, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 165      |
|    loss           | 8.23     |
|    neglogp        | 8.24     |
|    prob_true_act  | 0.000284 |
|    samples_so_far | 32       |
--------------------------------


309batch [00:01, 189.83batch/s]
490batch [00:02, 188.53batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00746 |
|    entropy        | 7.46     |
|    epoch          | 1        |
|    l2_loss        | 0        |
|    l2_norm        | 171      |
|    loss           | 5.36     |
|    neglogp        | 5.37     |
|    prob_true_act  | 0.00622  |
|    samples_so_far | 16032    |
--------------------------------


612batch [00:03, 191.33batch/s]
921batch [00:04, 177.16batch/s]
1000batch [00:05, 192.21batch/s]

...

14650batch [01:24, 175.93batch/s]
14969batch [01:27, 118.99batch/s]
14997batch [01:27, 125.88batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 15000    |
|    ent_loss       | 0.000317 |
|    entropy        | -0.317   |
|    epoch          | 48       |
|    l2_loss        | 0        |
|    l2_norm        | 231      |
|    loss           | 1.68     |
|    neglogp        | 1.68     |
|    prob_true_act  | 9.14     |
|    samples_so_far | 480032   |
--------------------------------


15276batch [01:29, 177.32batch/s]
15493batch [01:30, 190.54batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 15500    |
|    ent_loss       | 0.00033  |
|    entropy        | -0.33    |
|    epoch          | 49       |
|    l2_loss        | 0        |
|    l2_norm        | 232      |
|    loss           | -0.931   |
|    neglogp        | -0.931   |
|    prob_true_act  | 14.9     |
|    samples_so_far | 496032   |
--------------------------------


15594batch [01:30, 194.05batch/s]
15600batch [01:30, 171.94batch/s]


In [32]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_10.policy, env_10, 20, return_episode_rewards=True
)

In [33]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: 70.42540095000001 +/- 1.2229043436462428


In [34]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 2468.4317401499998 +/- 167.67637192521482


In [35]:
import torch
torch.save(bc_trainer_10.policy, "BC_MlpPolicy_Walker2D_10.pth.tar")